In [10]:
import sys
!{sys.executable} -m pip install numpy pandas ollama


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os, re, sys
import pandas as pd
import numpy as np

df=pd.read_csv(os.path.join(os.getcwd(),'input/gsc_queries.csv'))
df.head()

,Häufigste Suchanfragen,Klicks,Impressionen,CTR,Position
0,checkdomain dns einstellungen,5,51,9.8%,3.08
1,domainfactory dns einstellungen,3,97,3.09%,6.71
2,checkdomain dns einträge,3,66,4.55%,3.06
3,hosteurope nameserver ändern,2,97,2.06%,8.88
4,domainfactory dns settings,2,17,11.76%,6.00


In [2]:
df_qrs = df[['Häufigste Suchanfragen']][:30]
df_qrs = df_qrs.rename(columns={'Häufigste Suchanfragen': 'queries'})
df_qrs

,queries
0,checkdomain dns einstellungen
1,domainfactory dns einstellungen
2,checkdomain dns einträge
3,hosteurope nameserver ändern
4,domainfactory dns settings
5,strato ttl einstellen
6,checkdomain dns
7,domainfactory dns
8,rider proxy
9,google analytics einrichten


In [3]:
import ollama

model = 'avr/sfr-embedding-mistral'

def generate_embeddings(text, model=model):
    return ollama.embeddings(model=model, prompt=text)["embedding"]

df_qrs['embeddings'] = df_qrs["queries"].apply(lambda x : generate_embeddings (x, model = model))
# df_qrs.head()

In [ ]:
input = "was sind verschlüsselungstechniken?"

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model=model):
    return ollama.embed(input = [text], model=model)



def search_docs(df, user_query, top_n=1, to_print=True):
    embedding = get_embedding(
        user_query,
        model=model
    )['embeddings'][0]
    # print(embedding)
    df["similarities"] = df.embeddings.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

res = search_docs(df_qrs, input, top_n=10)['queries']
res.reset_index(drop=True, inplace=True)
# print(res)

,queries,embeddings,similarities
20,verschlüsselungstechniken,"[2.4645025730133057, 5.902427673339844, 1.1326...",0.738209
15,was ist server-seitiges tagging?,"[6.4108171463012695, 5.98980188369751, 5.92673...",0.723671
22,verschlüsselungsverfahren,"[2.7704415321350098, 4.128979206085205, 0.0157...",0.708248
10,datenverschlüsselung,"[4.057918548583984, 5.1801862716674805, -1.556...",0.687630
27,google tag manager einbinden,"[1.3264150619506836, 10.64944839477539, 0.3339...",0.529501
17,google analytics einbinden,"[0.04426678270101547, 10.260786056518555, 0.50...",0.525538
9,google analytics einrichten,"[0.2725484371185303, 9.54599666595459, 0.12453...",0.522087
13,datenschutzgesetze,"[4.50428581237793, 8.98518180847168, -4.265767...",0.518313
14,vorteile des serverseitigen taggings,"[3.2849016189575195, 7.259124279022217, 2.2316...",0.515796
11,google tag manager einrichten,"[0.7678918838500977, 8.936992645263672, 0.5328...",0.511966


In [8]:
output = ollama.generate(
    model='mistral',
    prompt=f"Based on the query: {input}, please provide a detailed response based on the following data: {res[1]}",
)
print(output['response'])

 To answer your original question, "Was sind Verschlüsselungstechniken?" (What are encryption techniques?), let me first explain what I understand you're asking about server-seitiges Tagging ("What is server-side tagging?").

Server-side tagging refers to a method used for categorizing content or data on a website, which is managed by the web server rather than through client-side scripts (such as JavaScript) in the user's browser. This approach can help improve site search functionality and allow for more accurate indexing of content.

Now, let's address your main question:

Verschlüsselungstechniken (Encryption Techniques) are methods used to secure sensitive data by converting it into an unreadable format, known as ciphertext, before transmitting or storing it. This ensures that only the intended recipient can access the original message, or plaintext, using a decryption key or algorithm. Encryption techniques can be applied in various ways depending on the context, such as:

1. Sym